In [ ]:
#무신사
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import Workbook
import time


options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
url = "https://www.musinsa.com/products/5086744"
driver.get(url)

try : 
    review_button = WebDriverWait(driver, 10). until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '스냅·후기')]"))
    )
    review_button.click()
    time.sleep(2)

except Exception as e : 
    print("리뷰 탭 클릭 실패 : ", e)


scroll_pause = 1
last_height =  driver.execute_script("return document.body.scrollHeight")

for _ in range(5) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height :
        break
    last_height = new_height 

soup = BeautifulSoup(driver.page_source, "html.parser")

product_name = soup.select_one("span.text-title_18px_med.sc-1omefes-1.exqQRL.font-pretendard").get_text(strip=True)

reviews_cards = soup.select("div.GoodsReviewStaticList__Container-sc-s1ucwv-0.lnqPxe")


reviews = []
for card in reviews_cards :
    text_tags = card.select("span.text-body_13px_reg.text-black.font-pretendard")
    for tag in text_tags :
        text = tag.get_text(strip=True)
        if text and len(text) > 10 :
            reviews.append(text)

reviews = list(set(reviews))

print(f"상품명 : {product_name}")
for idx, review in enumerate(reviews, start=1) :
    print(f"{idx}, {review}")

In [27]:
import requests
import urllib.parse
import pandas as pd
import re
from collections import Counter
from soynlp.tokenizer import RegexTokenizer

# 1. 네이버 API
client_id = "aO7WuC_fT8Ma_CCVOjZY"
client_pw = "J3YU12Svhb"

query = "아식스 운동화추천"
encoded_query = urllib.parse.quote(query)
naver_open_api = f"https://openapi.naver.com/v1/search/blog.json?query={encoded_query}&display=50"
header_parms = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_pw
}

res = requests.get(naver_open_api, headers=header_parms)
nike = res.json()

# 2. JSON 정리
items = nike.get('items', [])
df = pd.DataFrame(items)

# 3. HTML 태그 제거 및 한글 정제
def clean_text(text):
    text = re.sub(r'<.*?>', '', str(text))  # HTML 제거
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z\s]', '', text)  # 한글, 영문만
    return text

df['내용'] = df['description'].apply(clean_text)

# 4. soynlp로 토큰화
tokenizer = RegexTokenizer()
all_tokens = []

for text in df['내용']:
    tokens = tokenizer.tokenize(text)
    all_tokens.extend(tokens)

# 5. 불용어 제거 및 2글자 이상 필터링
stopwords = {
    "그리고", "그러나", "하지만", "그래서", "정말", "너무", "이런", "저런", "이것", "그것", 
    "있는", "없는", "하는", "해서", "한다", "했다", "합니다", "입니다", "그때", "그런", 
    "우선", "또한", "오늘은", "러닝화를", "러닝화는", "러닝화가", "월부터", "그냥", "이번엔"
}

filtered_tokens = [word for word in all_tokens if len(word) >= 2 and word not in stopwords]

# 6. 키워드 빈도 계산
counter = Counter(filtered_tokens)
df_keywords = pd.DataFrame(counter.items(), columns=["단어", "빈도"]).sort_values(by="빈도", ascending=False)

# 7. 결과 출력
print(df_keywords)

# 8. 엑셀로 저장
df_keywords.to_excel("아식스 운동화추천_주요_키워드.xlsx", index=False)

         단어  빈도
0       아식스  72
1       운동화  54
2        추천  35
25   아식스운동화  15
26       조그  12
..      ...  ..
671     경우는   1
672    두사이즈   1
673    하시기를   1
674    칼발이다   1
337    게시글은   1

[676 rows x 2 columns]
